# Analyse a Transmon Qubit using ElmerFEM
This notebook demonstrates the new open-source rendering and simulation capabilities of Qiskit Metal using `Gmsh` and `ElmerFEM` for tuning the performance parameters of a transmon qubit. instructions to download and install this branch of Qiskit Metal with Gmsh and ElmerFEM, can he found [here](https://github.com/Qiskit/qiskit-metal/blob/elmer_renderer/tutorials/4%20Analysis/B.%20Advanced%20-%20Direct%20use%20of%20the%20renderers/Gmsh-Elmer-install-instructions.md) The tutorial has the following steps:

## Contents
### 1. Creating a Transmon Qubit in Qiskit Metal
### 2. Rendering and meshing your design in the `QGmshRenderer`
- Rendering the design wireframe
- Applying a basic mesh
- Customizing the mesh
    - Setting initial mesh size parameters
    - Using the Intelli-mesh feature in `QGmshRenderer`
- Saving your mesh to a file

### 3. Rendering your design in `QElmerRenderer` (uses `QGmshRenderer`)
- Render design and generate mesh
- Export the mesh
- Run the capacitance simulation
    - Add solution setup
    - Run the solver
    - Export capacitance matrix

### 4. Perform LOM 2.0 Analysis
- Import the capacitance matrix
- Define cells and subsystems
- Define the composite system
- Compute the results:
    - Qubit Frequency ($f_Q$)
    - Anharmonicity ($\alpha$)
    - Readout $\chi$
    - Coupling $g$

# Necessary Imports

In [3]:
%load_ext autoreload
%autoreload 2

# Import basic things of Qiskit Metal
from qiskit_metal import MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6

# Import the Gmsh renderer
from qiskit_metal.renderers.renderer_gmsh.gmsh_renderer import QGmshRenderer

# Import the Elmer renderer
from qiskit_metal.renderers.renderer_elmer.elmer_renderer import QElmerRenderer
from qiskit_metal.renderers.renderer_elmer.elmer_renderer import load_capacitance_matrix_from_file

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%metal_heading 1. Create a Transmon Qubit in Qiskit Metal

In [5]:
# Instantiate a design object
design = designs.MultiPlanar({}, True)

# Invoke the Qiskit Metal GUI
gui = MetalGUI(design)

In [6]:
# Define a dictionary for connection pad options for the transmon
conn_pads = dict(
    connection_pads = dict(
        readout = dict(loc_W=0, loc_H=-1),
        coupler1 = dict(loc_W=-1, loc_H=1),
        coupler2 = dict(loc_W=1, loc_H=1)
    )
)

# Create a TransmonPocket6 object
q1 = TransmonPocket6(design, "Q1", options=dict(**conn_pads))

# Rebuild and autoscale the GUI
gui.rebuild()
gui.autoscale()

In [7]:
%metal_heading 2. Rendering and meshing your design in the `QGmshRenderer`

## Rendering the design wireframe

In [8]:
# Instantiate the Gmsh renderer
gmsh_renderer = QGmshRenderer(design)

# Render the design
# Tip: `mesh_geoms = False` will not mesh the design,
# but only draw the wire-frame of the geomtries
gmsh_renderer.render_design(open_pins=[("Q1", "coupler1"), ("Q1", "coupler2"), ("Q1", "readout")],
                            mesh_geoms=False)

In [9]:
# Launch Gmsh GUI to see the rendererd design
gmsh_renderer.launch_gui()

## Applying a basic mesh

In [10]:
# Add a basic mesh to the design and luanch the GUI to view
gmsh_renderer.add_mesh(dim=3, intelli_mesh=False)
gmsh_renderer.launch_gui()

## Customizing the mesh
### Setting initial mesh size parameters

In [11]:
# Update the mesh options
gmsh_renderer.options.mesh.min_size = '5um'
gmsh_renderer.options.mesh.max_size = '20um'

# Render the design
gmsh_renderer.render_design(open_pins=[("Q1", "coupler1"), ("Q1", "coupler2"), ("Q1", "readout")],
                            mesh_geoms=False)

# Mesh the design (without intelli-mesh)
gmsh_renderer.add_mesh(intelli_mesh=False)
gmsh_renderer.launch_gui()

### Using the Intelli-mesh feature in QGmshRenderer

In [19]:
# Update the mesh options
gmsh_renderer.options.mesh.min_size = '5um'
gmsh_renderer.options.mesh.max_size = '50um'

# Render and mesh the design (with intelli-mesh)
gmsh_renderer.render_design(open_pins=[("Q1", "coupler1"), ("Q1", "coupler2"), ("Q1", "readout")],
                            mesh_geoms=True)
gmsh_renderer.launch_gui()

## Saving your mesh to a file

In [12]:
# Export the Gmsh generated mesh to a file
gmsh_renderer.export_mesh("test.msh")

In [13]:
# Close the Gmsh renderer
gmsh_renderer.close()

True

In [14]:
%metal_heading 3. Rendering your design in `QElmerRenderer` (uses `QGmshRenderer`)

In [15]:
# Instantiate the Elmer renderer
elmer_renderer = QElmerRenderer(design)

# Elmer renderer uses the Gmsh renderer to
# generate a mesh for the design
# Set initial parameters for meshing in Gmsh (IMPORTANT step!!)
elmer_renderer.gmsh.options.mesh.min_size = "5um"
elmer_renderer.gmsh.options.mesh.max_size = "50um"

In [16]:
# Render the design
elmer_renderer.render_design(open_pins=[("Q1", "coupler1"), ("Q1", "coupler2"), ("Q1", "readout")],
                             skip_junctions=True)

In [17]:
# View the generated mesh
elmer_renderer.launch_gmsh_gui()

07:06PM 54s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png

07:06PM 54s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png



In [18]:
# Export the generated mesh
elmer_renderer.export_mesh()

In [19]:
# Add a solution setup to solve for the capacitance matrix
elmer_renderer.add_solution_setup('capacitance')

In [20]:
# Run the simulation in ElmerFEM
elmer_renderer.run('capacitance')

07:07PM 08s INFO [run]: Running ElmerGrid on input mesh from Gmsh...
07:07PM 09s INFO [run]: Running ElmerSolver for solver type: 'capacitance'


In [21]:
# Display the capacitnce matrix obtained after the simulation
elmer_renderer.capacitance_matrix

,Q1_readout_connector_pad,Q1_coupler1_connector_pad,Q1_coupler2_connector_pad,Q1_pad_top,Q1_pad_bot,ground_plane
Q1_readout_connector_pad,66.152393,-0.217088,-0.216448,-1.816632,-18.579164,-45.323061
Q1_coupler1_connector_pad,-0.217088,61.577912,-0.299855,-16.289710,-1.645005,-43.126254
Q1_coupler2_connector_pad,-0.216448,-0.299855,61.738637,-16.257792,-1.648347,-43.316194
Q1_pad_top,-1.816632,-16.289710,-16.257792,110.876880,-37.279421,-39.233324
Q1_pad_bot,-18.579164,-1.645005,-1.648347,-37.279421,104.174776,-45.022840
ground_plane,-45.323061,-43.126254,-43.316194,-39.233324,-45.022840,300.000000


In [22]:
# Run this, and in the Gmsh window,
# Right click on an empty area, click on "Toggle mesh visibility"
# Next, go to: Tools -> Visibility, select "Physical groups" in drop down menu
# Select "Volume 2", click on apply
# On the left pane, click on "Post-processing", and select the views that you want to observe
elmer_renderer.display_post_processing_data()

07:08PM 28s INFO [model]: Added new model 'post_processing' and set as current.
07:09PM 35s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png

07:09PM 35s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png



In [23]:
# Export the capacitance matrix
elmer_renderer.save_capacitance_matrix("cap_matrix.txt")

# Close the elmer renderer
elmer_renderer.close()

In [24]:
%metal_heading 4. Perform LOM 2.0 Analysis

In [25]:
# Import necessary things for LOM 2.0 Analysis
import numpy as np
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem 
from scipy.constants import speed_of_light as c_light
import matplotlib.pyplot as plt
%matplotlib inline

07:09PM 50s INFO [__init__]: TransmonBuilder with system_type TRANSMON registered to QuantumSystemRegistry
07:09PM 50s INFO [__init__]: FluxoniumBuilder with system_type FLUXONIUM registered to QuantumSystemRegistry
07:09PM 50s INFO [__init__]: TLResonatorBuilder with system_type TL_RESONATOR registered to QuantumSystemRegistry
07:09PM 50s INFO [__init__]: LumpedResonatorBuilder with system_type LUMPED_RESONATOR registered to QuantumSystemRegistry


In [26]:
# Load the saved capacitance matrix
cap_matrix = load_capacitance_matrix_from_file("cap_matrix.txt")

In [27]:
# Define cells
opt1 = dict(
    node_rename = {'Q1_coupler1_connector_pad': 'coupler1', 'Q1_coupler2_connector_pad': 'coupler2', 'Q1_readout_connector_pad': 'readout'}, 
    cap_mat = cap_matrix,
    ind_dict = {('Q1_pad_top', 'Q1_pad_bot'): 12},  # junction inductance in nH
    jj_dict = {('Q1_pad_top', 'Q1_pad_bot'): 'j1'},
    cj_dict = {('Q1_pad_top', 'Q1_pad_bot'): 2}, # junction capacitance in fF
)
cell_1 = Cell(opt1)

In [28]:
# Define subsystems
# subsystem 1: Transmon
transmon = Subsystem(name='transmon1_sys', sys_type='TRANSMON', nodes=['j1'])

# subsystem 2: Coupler 1
q_opts = dict(
    f_res = 7.4, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
coup1 = Subsystem(name='coup1_sys', sys_type='TL_RESONATOR', nodes=['coupler1'], q_opts=q_opts)

# subsystem 3: Coupler 2
q_opts = dict(
    f_res = 7.2, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
coup2 = Subsystem(name='coup2_sys', sys_type='TL_RESONATOR', nodes=['coupler2'], q_opts=q_opts)

# subsystem 4: Readout
q_opts = dict(
    f_res = 7, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
readout = Subsystem(name='readout_sys', sys_type='TL_RESONATOR', nodes=['readout'], q_opts=q_opts)

In [29]:
# Define the composite system
composite_sys = CompositeSystem(
    subsystems=[transmon, coup1, coup2, readout], 
    cells=[cell_1], 
    grd_node='ground_plane',
    nodes_force_keep=['coupler1', 'coupler2', 'readout']
)

In [30]:
# Get the circuit graph object
cg = composite_sys.circuitGraph()
print(cg)

node_jj_basis:
-------------

['j1', 'Q1_pad_bot', 'coupler1', 'coupler2', 'readout']

nodes_keep:
-------------

['j1', 'coupler1', 'coupler2', 'readout']


L_inv_k (reduced inverse inductance matrix):
-------------

                j1  coupler1  coupler2  readout
j1        0.083333       0.0       0.0      0.0
coupler1  0.000000       0.0       0.0      0.0
coupler2  0.000000       0.0       0.0      0.0
readout   0.000000       0.0       0.0      0.0

C_k (reduced capacitance matrix):
-------------

                 j1   coupler1   coupler2    readout
j1        74.322695  -6.894572  -6.877623   8.867749
coupler1  -6.894572  59.288442  -2.585677  -2.820728
coupler2  -6.877623  -2.585677  59.456458  -2.815940
readout    8.867749  -2.820728  -2.815940  63.191469




In [31]:
# Create a hilbert space with the composite system
hilbertspace = composite_sys.create_hilbertspace()
print(hilbertspace)

 c:\users\chris\qiskit-metal\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170
 c:\users\chris\qiskit-metal\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170
 c:\users\chris\qiskit-metal\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170


HilbertSpace:  subsystems
-------------------------

Transmon------------| [Transmon_1]
                    | EJ: 13621.792733898432
                    | EC: 270.57949289578147
                    | ng: 0.001
                    | ncut: 22
                    | truncated_dim: 10
                    |
                    | dim: 45


Oscillator----------| [Oscillator_1]
                    | E_osc: 7400.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3


Oscillator----------| [Oscillator_2]
                    | E_osc: 7200.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3


Oscillator----------| [Oscillator_3]
                    | E_osc: 7000
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3




In [32]:
# Add interaction between the subsystems
hilbertspace = composite_sys.add_interaction()

# Get the total hamiltonian of the composite system
hilbertspace.hamiltonian()

 c:\users\chris\qiskit-metal\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170
 c:\users\chris\qiskit-metal\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170
 c:\users\chris\qiskit-metal\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170


Quantum object: dims = [[10, 3, 3, 3], [10, 3, 3, 3]], shape = (270, 270), type = oper, isherm = True
Qobj data =
[[-10976.2043239 +0.00000000e+00j      0.        +9.72863864e-02j
       0.        +0.00000000e+00j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 [     0.        -9.72863864e-02j  -3976.2043239 +0.00000000e+00j
       0.        +1.37583727e-01j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 [     0.        +0.00000000e+00j      0.        -1.37583727e-01j
    3023.7956761 +0.00000000e+00j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 ...
 [     0.        +0.00000000e+00j      0.        +0.00000000e+00j
       0.        +0.00000000e+00j ...  57133.21414948+0.00000000e+00j
       0.        +4.68617745e+02j      0.        +0.00000000e+00j]
 [     0.        +0.00000000e+00j      0.        +0.00000000e+00j
   

In [33]:
# Get the reults from the hamiltonian
# Qubit frequency
# Chi matrix (having anharmonicity and readout chi values)
hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace, evals_count=30)


system frequencies in GHz:
--------------------------
{'transmon1_sys': 5.127861910526924, 'coup1_sys': 7.405303226589361, 'coup2_sys': 7.20137784091077, 'readout_sys': 7.004846291822175}

Chi matrices in MHz
--------------------------
               transmon1_sys  coup1_sys  coup2_sys  readout_sys
transmon1_sys    -305.406742  -1.451566  -1.020813    -2.451348
coup1_sys          -1.451566   2.111085   0.082769    -0.004390
coup2_sys          -1.020813   0.082769   2.027919     0.003114
readout_sys        -2.451348  -0.004390   0.003114     2.810141


In [34]:
chi_df = hamiltonian_results['chi_in_MHz'].to_dataframe()
print("Transmon frequency     :", hamiltonian_results['fQ_in_Ghz']['transmon1_sys'], "GHz")
print("Transmon Anharmonicity :", chi_df['transmon1_sys']['transmon1_sys'], "MHz")
print("Readou chi             :", chi_df['readout_sys']['transmon1_sys'], "MHz")

Transmon frequency     : 5.127861910526924 GHz
Transmon Anharmonicity : -305.40674235535334 MHz
Readou chi             : -2.451347652295226 MHz


In [35]:
# Get the coupling 'g' values between qubit and resonators
composite_sys.compute_gs().to_dataframe()

,j1,coupler1,coupler2,readout
j1,0.000000,85.727494,83.271172,-97.286513
coupler1,85.727494,0.000000,17.162605,10.710770
coupler2,83.271172,17.162605,0.000000,10.414323
readout,-97.286513,10.710770,10.414323,0.000000


In [36]:
# Hamiltonian parameters for the transmon
transmon.h_params

defaultdict(dict,
            {'j1': {'EJ': 13621.792733898432,
              'EC': 270.57949289578147,
              'Q_zpf': 3.204353268e-19,
              'default_charge_op': Operator(op=array([[-22,   0,   0, ...,   0,   0,   0],
                     [  0, -21,   0, ...,   0,   0,   0],
                     [  0,   0, -20, ...,   0,   0,   0],
                     ...,
                     [  0,   0,   0, ...,  20,   0,   0],
                     [  0,   0,   0, ...,   0,  21,   0],
                     [  0,   0,   0, ...,   0,   0,  22]]), add_hc=False)}})

In [37]:
# Close the Qiskit Metal GUI
gui.main_window.close()

True